In [5]:
#Import nessesary libraries

import pandas as pd
import sqlite3
from urllib.request import urlopen
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
from itertools import permutations

# pip install geopy #This is used to calculate distance based on lat and long... no need to reinvent the wheel...

from geopy.distance import great_circle



In [4]:
#Read in top airport data from supplied files
path = 'hw_5_data/'

top_airpots = pd.read_csv(path+'top_airports.csv' )
airport_info = pd.read_csv(path+'ICAO_airports.csv' )



In [1]:

airport_info['ICAO'] = airport_info['ident']
combined_info = top_airpots.merge(airport_info, on='ICAO')




NameError: name 'airport_info' is not defined

In [2]:
def get_data_from_db(db_name, path, sql_cmd):
        connection = sqlite3.connect(path+db_name)
        cursor = connection.cursor()
        connection.row_factory = lambda cursor, row: row[0]
        c = connection.cursor()
        data = c.execute(sql_cmd).fetchall()
        return data
    

In [123]:
column_names = ['Airport_pairs','highT_1','highT_3','highT_7',\
                'precip_1','precip_3','precip_7','dist','long_delta' ]
#Create dictionary to hold all the correlation data
airport_pairs_dict = {}

for key in column_names: 
    airport_pairs_dict[key] = list()
    
    
#pairs_df = pd.DataFrame(columns = column_names)

#airport_code = []


#connection = sqlite3.connect(path+'airports2.db')

cursor = connection.cursor()

sql_cmd = "SELECT ICAO FROM airport_info"
cursor.execute(sql_cmd)

connection.row_factory = lambda cursor, row: row[0]
c = connection.cursor()
airport_code = c.execute(sql_cmd).fetchall()



    
connection.commit()
#connection.close()

indx = 0

bad_airport = np.array(['KIAH','KLGA','KMSY','KIND'])
airport_code = np.asarray(airport_code)
cuts = np.ones(shape=airport_code.shape, dtype=bool)
bad_indx = np.where(airport_code == 'KIAH')#np.array([word for word in bad_airport]))
cuts[bad_indx] = 0
bad_indx = np.where(airport_code == 'KLGA')#[word for word in bad_airport])
cuts[bad_indx] = 0
bad_indx = np.where(airport_code == 'KMSY')#[word for word in bad_airport])
cuts[bad_indx] = 0
bad_indx = np.where(airport_code == 'KIND')#[word for word in bad_airport])
cuts[bad_indx] = 0



for combo in permutations(airport_code[cuts],2):
    
    #print(combo)
    indx += 1
    
    sql_cmd_highT_1 = "SELECT high_temp FROM airport_weather where ICAO = " + '"' + str(combo[0] + '"')
    sql_cmd_highT_2 = "SELECT high_temp FROM airport_weather where ICAO = " + '"' + str(combo[1] + '"')
    
    sql_cmd_percip_1 = "SELECT precipitation FROM airport_weather where ICAO = " + '"' + str(combo[0] + '"')
    sql_cmd_percip_2 = "SELECT precipitation FROM airport_weather where ICAO = " + '"' + str(combo[1] + '"')
    
    sql_cmd_lat_1 = "SELECT latitude_deg FROM airport_info where ICAO = " + '"' + str(combo[0] + '"')
    sql_cmd_lat_2 = "SELECT latitude_deg FROM airport_info where ICAO = " + '"' + str(combo[1] + '"')
    
    sql_cmd_long_1 = "SELECT longitude_deg FROM airport_info where ICAO = " + '"' + str(combo[0] + '"')
    sql_cmd_long_2 = "SELECT longitude_deg FROM airport_info where ICAO = " + '"' + str(combo[1] + '"')
    
    #connection.row_factory = lambda cursor, row: row[0]
    #c = connection.cursor()
    high_temp_1 = c.execute(sql_cmd_highT_1).fetchall()
    high_temp_2 = c.execute(sql_cmd_highT_2).fetchall()
    
    precip_1 = c.execute(sql_cmd_percip_1).fetchall()
    precip_2 = c.execute(sql_cmd_percip_2).fetchall()
    
    lat_1 = c.execute(sql_cmd_lat_1).fetchall()[0]
    lat_2 = c.execute(sql_cmd_lat_2).fetchall()[0]
    
    long_1 = c.execute(sql_cmd_long_1).fetchall()[0]
    long_2 = c.execute(sql_cmd_long_2).fetchall()[0]
    
    
    one_highT_coeff = np.corrcoef(high_temp_1[:-1],high_temp_2[1:])[0,1]
    three_highT_coeff = np.corrcoef(high_temp_1[:-3],high_temp_2[3:])[0,1]
    seven_highT_coeff = np.corrcoef(high_temp_1[:-7],high_temp_2[7:])[0,1]
    
    one_precip_coeff = np.corrcoef(high_temp_1[:-1],high_temp_2[1:])[0,1]
    three_precip_coeff = np.corrcoef(high_temp_1[:-3],high_temp_2[3:])[0,1]
    seven_precip_coeff = np.corrcoef(high_temp_1[:-7],high_temp_2[7:])[0,1]
    
    long_delta = abs(long_1-long_2)
    
    distance = great_circle((lat_1,long_1),(lat_2,long_2)).miles
    

    airport_pairs_dict['Airport_pairs'].append(combo)
    airport_pairs_dict['highT_1'].append(one_highT_coeff)
    airport_pairs_dict['highT_3'].append(three_highT_coeff)
    airport_pairs_dict['highT_7'].append(seven_highT_coeff)
    airport_pairs_dict['precip_1'].append(one_precip_coeff)
    airport_pairs_dict['precip_3'].append(three_precip_coeff)
    airport_pairs_dict['precip_7'].append(seven_precip_coeff)
    airport_pairs_dict['dist'].append(distance)
    airport_pairs_dict['long_delta'].append(long_delta)
    
    
pairs_df = pd.DataFrame.from_dict(airport_pairs_dict).set_index('Airport_pairs') 

    
    

    
    

In [129]:

pairs_df = pd.DataFrame.from_dict(airport_pairs_dict).set_index('Airport_pairs') 

In [131]:
pairs_df



,dist,highT_1,highT_3,highT_7,long_delta,precip_1,precip_3,precip_7
Airport_pairs,,,,,,,,
"(KATL, KORD)",606.832917,0.799438,0.769494,0.764711,3.476700,0.799438,0.769494,0.764711
"(KATL, KLAX)",1942.814738,0.303210,0.335047,0.363819,33.979897,0.303210,0.335047,0.363819
"(KATL, KDFW)",730.025887,0.770481,0.742206,0.745685,12.609901,0.770481,0.742206,0.745685
"(KATL, KDEN)",1197.482139,0.666448,0.690823,0.702475,20.244896,0.666448,0.690823,0.702475
"(KATL, KJFK)",759.587605,0.857789,0.802000,0.794981,10.649200,0.857789,0.802000,0.794981
"(KATL, KSFO)",2134.836180,0.519841,0.535392,0.553202,37.946899,0.519841,0.535392,0.553202
"(KATL, KLAS)",1743.330326,0.753782,0.772790,0.774858,30.723900,0.753782,0.772790,0.774858
"(KATL, KPHX)",1584.371935,0.737779,0.762787,0.765186,27.583900,0.737779,0.762787,0.765186
"(KATL, KCLT)",226.595296,0.906927,0.794965,0.758109,3.485001,0.906927,0.794965,0.758109


In [6]:
path = 'hw_5_data/'
sql_cmd = "SELECT ICAO FROM airport_info"
a = get_data_from_db('airports2.db', path, sql_cmd)
a

['KATL',
 'KORD',
 'KLAX',
 'KDFW',
 'KDEN',
 'KJFK',
 'KSFO',
 'KIAH',
 'KLAS',
 'KPHX',
 'KCLT',
 'KMIA',
 'KMCO',
 'KEWR',
 'KDTW',
 'KMSP',
 'KSEA',
 'KPHL',
 'KBOS',
 'KLGA',
 'KIAD',
 'KBWI',
 'KFLL',
 'KSLC',
 'PHNL',
 'KDCA',
 'KMDW',
 'KSAN',
 'KTPA',
 'KPDX',
 'KSTL',
 'KMCI',
 'KMEM',
 'KCLE',
 'KOAK',
 'TJSJ',
 'KRDU',
 'KBNA',
 'KSMF',
 'KHOU',
 'KSNA',
 'KAUS',
 'KSJC',
 'KMSY',
 'KPIT',
 'KSAT',
 'KCVG',
 'KMKE',
 'KDAL',
 'KIND']